In [ ]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re

In [ ]:
# Brandon's code starts here
csv_file = "Resources/dnd_monsters.csv"
dndmonster_df= pd.read_csv(csv_file)
dndmonster_df.head()

In [ ]:
clean_monster_df = dndmonster_df[["name","cr","type","size","ac","hp","align"]]
clean_monster_df.head()

In [ ]:
clean_monster_df.dropna()

In [ ]:
for index, row in clean_monster_df.iterrows():
    if isinstance(row.cr, str):
        row= row.copy()
        target = 

In [ ]:
clean_monster_df.dtypes

In [ ]:
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/4","0.25", str(s)))
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/2","0.50", str(s)))
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/8","0.125", str(s)))

In [ ]:
clean_monster_df["cr"] = clean_monster_df["cr"].astype(float)

In [ ]:
clean_monster_df.dtypes

In [ ]:
# Brandon's code ends here

In [ ]:
# Mauvonte's code starts here

In [ ]:
# Mauvonte's code ends here

In [ ]:
# Scott's code starts here

In [ ]:
# Scott's code ends here